In [43]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score, mean_squared_error, r2_score
import statistics
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from keras.optimizers import SGD, Adam, RMSprop, Adamax

import numpy as np
import pandas as pd

In [71]:
df = pd.read_csv("../df.csv")
doc2vec50 = pd.read_csv("../dataset/doc2vec_50.csv")
df_train= df[["essay_set", "essay","total_score", "word_count","Mistakes","reading_ease"]]
df_train = doc2vec50
df_train = pd.concat([df_train, df[["essay_set", "word_count","Mistakes","reading_ease"]]], axis = 1, join = "inner").drop(['Unnamed: 0'], axis = 1)
y = df["total_score"]
X = df_train

In [29]:
df = pd.read_csv("../df.csv")
doc2vec300 = pd.read_csv("../dataset/doc2vec_300.csv")
df_train= df[["essay_set", "essay","total_score", "word_count","Mistakes","reading_ease"]]
df_train = doc2vec300
df_train = pd.concat([df_train, df[["essay_set", "word_count","Mistakes","reading_ease"]]], axis = 1, join = "inner").drop(['Unnamed: 0'], axis = 1)
y = df["total_score"].values
X = df_train.values


In [75]:
def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(54, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 54], return_sequences=True))
    model.add(LSTM(54, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model

In [73]:
# doc_emb

In [ ]:
cv = KFold(n_splits = 5, shuffle = True)
results = []
r2s = []
mses = []
y_pred_list = []
count = 1

for num in range(1,9):
    y = df.loc[df_train["essay_set"] == num]["total_score"]
    X = df_train.loc[df_train["essay_set"] == num]
    print("\n###########Set-{}###########\n".format(num))
    count = 1
    result_for_set = []
    for traincv, testcv in cv.split(X):
        print("\n--------Fold {}--------\n".format(count))
        
        X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
        X_train = X_train.to_numpy()
        X_test = X_test.to_numpy()

        X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

        lstm_model = get_model()
        lstm_model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)


        y_pred_temp = lstm_model.predict(X_test)

        y_pred = []
        for i in y_pred_temp:
            y_pred.append(int(round(i[0])))
        y_pred_temp = np.around(y_pred_temp)
        if count == 5:
             lstm_model.save_weights('final_lstm.h5')
        y_test_new = []
        for  i in list(y_test.array):
            y_test_new.append(int(i))
        result = cohen_kappa_score(y_test_new,y_pred,weights='quadratic')
        print("Kappa Score: {}".format(result))
        result_for_set.append(result)
        count += 1
    results.append(result_for_set)


###########Set-1###########


--------Fold 1--------

Model: "sequential_117"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_227 (LSTM)              (None, 1, 54)             23544     
_________________________________________________________________
lstm_228 (LSTM)              (None, 54)                23544     
_________________________________________________________________
dropout_117 (Dropout)        (None, 54)                0         
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 55        
Total params: 47,143
Trainable params: 47,143
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
36/36 [==============================] - 1s 28ms/step - loss: 61.7356 - mae: 7.6710 - val_loss: 32.7351 - val_mae: 5.5133
Epoch 2/50
36/36 [==============================] - 0s 5ms

36/36 [==============================] - 1s 29ms/step - loss: 58.7191 - mae: 7.4688 - val_loss: 26.5851 - val_mae: 4.9555
Epoch 2/50
36/36 [==============================] - 0s 5ms/step - loss: 22.1634 - mae: 4.1789 - val_loss: 3.8980 - val_mae: 1.6394
Epoch 3/50
36/36 [==============================] - 0s 5ms/step - loss: 5.1716 - mae: 1.8132 - val_loss: 2.0403 - val_mae: 1.1074
Epoch 4/50
36/36 [==============================] - 0s 5ms/step - loss: 3.7696 - mae: 1.5479 - val_loss: 1.8996 - val_mae: 1.0590
Epoch 5/50
36/36 [==============================] - 0s 4ms/step - loss: 3.3402 - mae: 1.4319 - val_loss: 1.5043 - val_mae: 0.9173
Epoch 6/50
36/36 [==============================] - 0s 4ms/step - loss: 3.3121 - mae: 1.4352 - val_loss: 1.3923 - val_mae: 0.9075
Epoch 7/50
36/36 [==============================] - 0s 5ms/step - loss: 3.1360 - mae: 1.4007 - val_loss: 1.0911 - val_mae: 0.7825
Epoch 8/50
36/36 [==============================] - 0s 5ms/step - loss: 3.0253 - mae: 1.3712 - va

36/36 [==============================] - 0s 5ms/step - loss: 3.4665 - mae: 1.4917 - val_loss: 1.9561 - val_mae: 1.0696
Epoch 7/50
36/36 [==============================] - 0s 5ms/step - loss: 3.3813 - mae: 1.4363 - val_loss: 1.6392 - val_mae: 0.9516
Epoch 8/50
36/36 [==============================] - 0s 4ms/step - loss: 3.2321 - mae: 1.4151 - val_loss: 1.4944 - val_mae: 0.9131
Epoch 9/50
36/36 [==============================] - 0s 4ms/step - loss: 3.2574 - mae: 1.4161 - val_loss: 1.2142 - val_mae: 0.8392
Epoch 10/50
36/36 [==============================] - 0s 5ms/step - loss: 2.7979 - mae: 1.3382 - val_loss: 1.2161 - val_mae: 0.8443
Epoch 11/50
36/36 [==============================] - 0s 5ms/step - loss: 2.9042 - mae: 1.3560 - val_loss: 1.0380 - val_mae: 0.7898
Epoch 12/50
36/36 [==============================] - 0s 4ms/step - loss: 2.7036 - mae: 1.2986 - val_loss: 1.0284 - val_mae: 0.7747
Epoch 13/50
36/36 [==============================] - 0s 5ms/step - loss: 2.7424 - mae: 1.3030 - va

36/36 [==============================] - 0s 5ms/step - loss: 2.7340 - mae: 1.3079 - val_loss: 1.0463 - val_mae: 0.7902
Epoch 12/50
36/36 [==============================] - 0s 5ms/step - loss: 2.6781 - mae: 1.2954 - val_loss: 1.0473 - val_mae: 0.7812
Epoch 13/50
36/36 [==============================] - 0s 5ms/step - loss: 2.7796 - mae: 1.3077 - val_loss: 0.8608 - val_mae: 0.7227
Epoch 14/50
36/36 [==============================] - 0s 5ms/step - loss: 2.8985 - mae: 1.3568 - val_loss: 0.9478 - val_mae: 0.7504
Epoch 15/50
36/36 [==============================] - 0s 5ms/step - loss: 2.6703 - mae: 1.2845 - val_loss: 0.8607 - val_mae: 0.7173
Epoch 16/50
36/36 [==============================] - 0s 5ms/step - loss: 2.6485 - mae: 1.2849 - val_loss: 1.0955 - val_mae: 0.8133
Epoch 17/50
36/36 [==============================] - 0s 5ms/step - loss: 2.6351 - mae: 1.2796 - val_loss: 0.9644 - val_mae: 0.7757
Epoch 18/50
36/36 [==============================] - 0s 5ms/step - loss: 2.5466 - mae: 1.2670 -

36/36 [==============================] - 0s 4ms/step - loss: 2.7053 - mae: 1.2735 - val_loss: 1.1896 - val_mae: 0.8938
Epoch 17/50
36/36 [==============================] - 0s 4ms/step - loss: 2.6198 - mae: 1.2837 - val_loss: 0.7648 - val_mae: 0.6714
Epoch 18/50
36/36 [==============================] - 0s 4ms/step - loss: 2.5450 - mae: 1.2681 - val_loss: 0.8609 - val_mae: 0.7449
Epoch 19/50
36/36 [==============================] - 0s 4ms/step - loss: 2.4400 - mae: 1.2434 - val_loss: 0.8133 - val_mae: 0.7222
Epoch 20/50
36/36 [==============================] - 0s 4ms/step - loss: 2.5276 - mae: 1.2499 - val_loss: 1.1735 - val_mae: 0.8946
Epoch 21/50
36/36 [==============================] - 0s 4ms/step - loss: 2.6338 - mae: 1.2984 - val_loss: 0.7558 - val_mae: 0.6835
Epoch 22/50
36/36 [==============================] - 0s 4ms/step - loss: 2.4476 - mae: 1.2316 - val_loss: 1.0409 - val_mae: 0.8220
Epoch 23/50
36/36 [==============================] - 0s 4ms/step - loss: 2.6632 - mae: 1.2814 -

...

36/36 [==============================] - 0s 4ms/step - loss: 0.6705 - mae: 0.6482 - val_loss: 0.3878 - val_mae: 0.4875
Epoch 17/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6840 - mae: 0.6624 - val_loss: 0.3757 - val_mae: 0.4831
Epoch 18/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6292 - mae: 0.6336 - val_loss: 0.3754 - val_mae: 0.4847
Epoch 19/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6731 - mae: 0.6510 - val_loss: 0.3891 - val_mae: 0.4972
Epoch 20/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6301 - mae: 0.6310 - val_loss: 0.3880 - val_mae: 0.5021
Epoch 21/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6345 - mae: 0.6373 - val_loss: 0.3790 - val_mae: 0.4882
Epoch 22/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6278 - mae: 0.6362 - val_loss: 0.3873 - val_mae: 0.4950
Epoch 23/50
36/36 [==============================] - 0s 4ms/step - loss: 0.5994 - mae: 0.6138 -

36/36 [==============================] - 0s 5ms/step - loss: 0.6372 - mae: 0.6463 - val_loss: 0.3384 - val_mae: 0.4622
Epoch 22/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6210 - mae: 0.6253 - val_loss: 0.3416 - val_mae: 0.4635
Epoch 23/50
36/36 [==============================] - 0s 4ms/step - loss: 0.5640 - mae: 0.5953 - val_loss: 0.3918 - val_mae: 0.5080
Epoch 24/50
36/36 [==============================] - 0s 5ms/step - loss: 0.6012 - mae: 0.6260 - val_loss: 0.3482 - val_mae: 0.4758
Epoch 25/50
36/36 [==============================] - 0s 4ms/step - loss: 0.6354 - mae: 0.6377 - val_loss: 0.4141 - val_mae: 0.5274
Epoch 26/50
36/36 [==============================] - 0s 4ms/step - loss: 0.5931 - mae: 0.6175 - val_loss: 0.3361 - val_mae: 0.4638
Epoch 27/50
36/36 [==============================] - 0s 4ms/step - loss: 0.5790 - mae: 0.6070 - val_loss: 0.3277 - val_mae: 0.4567
Epoch 28/50
36/36 [==============================] - 0s 5ms/step - loss: 0.5816 - mae: 0.6124 -

Epoch 26/50
32/32 [==============================] - 0s 5ms/step - loss: 15.8814 - mae: 3.2014 - val_loss: 10.8837 - val_mae: 2.5706
Epoch 27/50
32/32 [==============================] - 0s 6ms/step - loss: 15.5744 - mae: 3.1006 - val_loss: 9.8465 - val_mae: 2.4300
Epoch 28/50
32/32 [==============================] - 0s 6ms/step - loss: 15.9084 - mae: 3.1550 - val_loss: 9.1711 - val_mae: 2.4535
Epoch 29/50
32/32 [==============================] - 0s 6ms/step - loss: 14.2808 - mae: 3.0425 - val_loss: 10.0775 - val_mae: 2.4811
Epoch 30/50
32/32 [==============================] - 0s 5ms/step - loss: 14.3666 - mae: 3.0435 - val_loss: 8.9469 - val_mae: 2.3683
Epoch 31/50
32/32 [==============================] - 0s 6ms/step - loss: 15.1125 - mae: 3.0991 - val_loss: 8.4005 - val_mae: 2.2768
Epoch 32/50
32/32 [==============================] - ETA: 0s - loss: 14.7990 - mae: 3.10 - 0s 6ms/step - loss: 15.0013 - mae: 3.1077 - val_loss: 8.9998 - val_mae: 2.4104
Epoch 33/50
32/32 [=================

32/32 [==============================] - 0s 5ms/step - loss: 15.9522 - mae: 3.2256 - val_loss: 12.9184 - val_mae: 2.8832
Epoch 31/50
32/32 [==============================] - 0s 5ms/step - loss: 16.4008 - mae: 3.2116 - val_loss: 10.6753 - val_mae: 2.5835
Epoch 32/50
32/32 [==============================] - 0s 5ms/step - loss: 15.4730 - mae: 3.1281 - val_loss: 11.1793 - val_mae: 2.6353
Epoch 33/50
32/32 [==============================] - 0s 5ms/step - loss: 15.4856 - mae: 3.1108 - val_loss: 10.3536 - val_mae: 2.5175
Epoch 34/50
32/32 [==============================] - 0s 5ms/step - loss: 15.2373 - mae: 3.1250 - val_loss: 9.7368 - val_mae: 2.4273
Epoch 35/50
32/32 [==============================] - 0s 5ms/step - loss: 15.6147 - mae: 3.1739 - val_loss: 10.0394 - val_mae: 2.4972
Epoch 36/50
32/32 [==============================] - 0s 5ms/step - loss: 15.8914 - mae: 3.1967 - val_loss: 9.6187 - val_mae: 2.3781
Epoch 37/50
32/32 [==============================] - 0s 5ms/step - loss: 14.7704 - 

Epoch 35/50
32/32 [==============================] - 0s 4ms/step - loss: 15.5611 - mae: 3.1846 - val_loss: 10.5343 - val_mae: 2.5834
Epoch 36/50
32/32 [==============================] - 0s 5ms/step - loss: 14.8460 - mae: 3.1112 - val_loss: 10.9778 - val_mae: 2.6057
Epoch 37/50
32/32 [==============================] - 0s 5ms/step - loss: 14.9057 - mae: 3.0977 - val_loss: 9.4770 - val_mae: 2.4433
Epoch 38/50
32/32 [==============================] - 0s 5ms/step - loss: 14.7869 - mae: 3.1046 - val_loss: 9.1603 - val_mae: 2.3913
Epoch 39/50
32/32 [==============================] - 0s 4ms/step - loss: 13.9582 - mae: 3.0117 - val_loss: 10.7261 - val_mae: 2.5717
Epoch 40/50
32/32 [==============================] - 0s 5ms/step - loss: 14.3872 - mae: 3.0218 - val_loss: 8.8183 - val_mae: 2.3449
Epoch 41/50
32/32 [==============================] - 0s 5ms/step - loss: 14.7532 - mae: 3.0932 - val_loss: 12.3155 - val_mae: 2.7785
Epoch 42/50
32/32 [==============================] - 0s 5ms/step - loss:

<generator object _BaseKFold.split at 0x0000020F720AD430>


In [35]:
X.to_numpy()

array([[-6.53643310e-01,  1.22631383e+00,  1.49060738e+00, ...,
         3.38000000e+02,  2.00000000e+01,  7.40200000e+01],
       [ 1.78830183e+00, -5.99211633e-01, -2.10853004e+00, ...,
         4.19000000e+02,  3.30000000e+01,  6.70800000e+01],
       [-2.45267693e-02,  5.21169484e-01, -8.56822908e-01, ...,
         2.79000000e+02,  2.00000000e+01,  6.82000000e+01],
       ...,
       [ 1.19136560e+00,  9.73212183e-01, -5.47442257e-01, ...,
         8.18000000e+02,  8.20000000e+01,  6.92500000e+01],
       [ 4.18220490e-01,  1.79089689e+00,  5.47983468e-01, ...,
         5.94000000e+02,  3.30000000e+01,  7.20500000e+01],
       [-8.03048015e-02,  1.26649606e+00,  1.87341690e-01, ...,
         4.68000000e+02,  1.10000000e+01,  7.20500000e+01]])

In [55]:
y = df.loc[df["essay_set"] == j]["total_score"]
X = df_train.loc[df_train["essay_set"] == j]


In [56]:
y

0        8.0
1        9.0
2        7.0
3       10.0
4        8.0
        ... 
1778     8.0
1779     7.0
1780     8.0
1781     2.0
1782     7.0
Name: total_score, Length: 1783, dtype: float64

In [57]:
X

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,essay_set,word_count,Mistakes,reading_ease
0,-0.653643,1.226314,1.490607,-0.273509,-0.126667,1.948407,-0.800371,-0.520892,-1.989100,0.305569,...,1.436761,1.510649,-0.300369,-0.430754,-1.164265,0.293608,1,338,20.0,74.02
1,1.788302,-0.599212,-2.108530,0.105541,-0.419768,0.424503,0.306117,-1.297395,-0.365960,-0.193469,...,-1.120824,0.798896,-0.735984,-1.692172,-0.989390,1.180730,1,419,33.0,67.08
2,-0.024527,0.521169,-0.856823,-0.467126,-0.856623,0.542416,-1.578355,-0.025513,-1.748438,-0.108801,...,-0.789562,0.019656,-1.198619,1.141882,-0.438716,-0.378533,1,279,20.0,68.20
3,2.232087,0.245010,-0.081397,2.165611,-1.296768,0.386925,-1.609302,0.097405,-2.632485,0.009429,...,-0.900712,1.323254,-1.008070,-2.154635,1.147291,1.223645,1,524,68.0,53.34
4,-0.532192,-0.046631,-1.994325,0.103210,-1.187608,0.603498,0.119099,0.494659,-0.015966,1.025912,...,0.127415,-0.186732,-2.030141,-2.104558,0.112237,-0.022088,1,465,23.0,72.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1778,0.076996,1.060306,-0.374213,-0.734702,0.987911,3.004169,1.927672,0.183494,-2.541197,0.138758,...,0.343130,0.955492,0.042598,-0.262129,-1.845663,-2.069562,1,497,46.0,77.30
1779,0.526733,2.083623,-1.318357,0.365669,0.885012,1.430439,-0.444463,0.419993,-1.443482,0.104980,...,1.393086,1.244183,-1.869425,-1.892759,-1.115894,-0.072377,1,200,17.0,84.88
1780,0.258367,0.962929,1.196177,-0.475155,-1.180123,2.435375,-0.833579,0.755691,0.030383,1.244768,...,-0.655146,-0.589399,-1.204976,1.377089,0.634938,-0.676763,1,291,14.0,71.04
1781,0.092214,0.710328,0.870210,-0.486119,-0.559627,0.609152,0.108019,0.114833,-0.310847,0.426044,...,0.377715,0.458104,-0.948134,-0.174676,-0.203892,-0.439191,1,15,0.0,90.09
